In [16]:
import sys
sys.path.append('/home/xinyiz/pamrats')

import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp


# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances




In [24]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
ifplot=True
ifcluster=True
inverseAct=True

distThreshAsMin=[2,4,6,8,10]

plottype='umap'
minCells=15 #min number of cells for analysis
# clustermethod=['kmeanbatch']
clustermethod=['leiden','dbscan','agglomerative','kmeanbatch']
#umap/leiden clustering parameters
n_neighbors=10
min_dist=0.25
n_pcs=40 #for clustering
# resolution=[0.5,1,1.5,2]
resolution=[0.05,0.1,0.2,0.3]
plotepoch=9990
savenameAdd=''
#DBscan
epslist= [6,8,10]
min_sampleslist=[15,30,45] 
#agglomerative
nclusterlist=[2,3,4,5,8,10]
aggMetric=['euclidean']


combineCelltype={'glia':['Astro','Micro', 'OPC', 'Oligo'],'CA':['CA1', 'CA2', 'CA3']}

maskedgeName='knn20_connectivity'

plot_samples={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
plot_sample_X=['logminmax']
# plot_sample_X=['corrected','scaled']
standardizeX=False
name='allk20XA_01_dca'
logsavepath='/mnt/xinyi/pamrats/log/train_gae_starmap/'+name
modelsavepath='/mnt/xinyi/pamrats/models/train_gae_starmap/'+name
plotsavepath='/mnt/xinyi/pamrats/plots/train_gae_starmap/'+name
    

In [18]:
#Load data
savedir=os.path.join('/mnt/xinyi/','starmap')
adj_dir=os.path.join(savedir,'a')
scaleddata=scanpy.read_h5ad('/mnt/xinyi/2021-01-13-mAD-test-dataset/2020-12-27-starmap-mAD-raw.h5ad')

adj_list={}
adj_list['disease13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9494.npz'))
adj_list['control13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9498.npz'))
adj_list['disease8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9723.npz'))
adj_list['control8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9735.npz'))

In [66]:
def plotNeighborComposition(clusterlabels,allCTlist,adjCluster,allDist,distThresh,savepath,savename):
    for cl in np.unique(clusterlabels):
        clIdx=(clusterlabels==cl)
        adjCluster_cl=np.copy(adjCluster[clIdx])
        allDist_cl=np.copy(allDist[clIdx])
        neighborctlist=[]
        neighborDistlist=[]
        for clm in range(np.sum(clIdx)):
            clm_nIdx=(adjCluster_cl[clm]!=0)
            clm_nCT=allCTlist[clm_nIdx]
            clm_nDist=allDist_cl[clm,clm_nIdx]
            neighborctlist=np.concatenate((neighborctlist,clm_nCT))
            neighborDistlist=np.concatenate((neighborDistlist,clm_nDist))
        nCT_unique,nCT_counts=np.unique(neighborctlist,return_counts=True)

        plt.bar(np.arange(nCT_unique.size), nCT_counts/np.sum(clIdx))
        plt.ylabel("counts/cell in cluster")
        plt.xticks(np.arange(nCT_unique.size), nCT_unique)
        plt.savefig(os.path.join(savepath,str(cl)+savename+'.jpg'))
        plt.close('all')
        
        ind = np.arange(len(distThresh)) 
        prebar=np.zeros(len(distThresh))
        for nctIdx in range(nCT_unique.size):
            nCT=nCT_unique[nctIdx]
            nCT_countsDist=[]
            nCTdistlist=neighborDistlist[neighborctlist==nCT]
            
            preDist=0
            for d in distThresh:
                nCT_countsDist.append(np.sum(np.logical_and(nCTdistlist>preDist,nCTdistlist<=d)))
                preDist=d
                
            if savename=='celltype_broad':
                plt.bar(ind, nCT_countsDist/np.sum(clIdx),color=plt.get_cmap('tab20')(np.where(np.unique(celltype_broad)==nCT)[0][0]),label=nCT,bottom=prebar)
            else:
                plt.bar(ind, nCT_countsDist/np.sum(clIdx),label=nCT,bottom=prebar)
            prebar+=nCT_countsDist/np.sum(clIdx)
            plt.ylabel('counts/cell in cluster')

        plt.xticks(ind, distThresh)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),ncol=6)
        plt.savefig(os.path.join(savepath,str(cl)+savename+'_byDist.jpg'),bbox_inches='tight')
        plt.close('all')
        
def plotNLeiden(allCTlist_dict,adjCluster,allDist,distThresh,savepath,n_neighbors,n_pcs,min_dist,resolution):
    for r in resolution:
        savenamecluster='leiden_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'n_pcs'+str(n_pcs)+'res'+str(r)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        savepathR=os.path.join(savepath,savenamecluster)
        if not os.path.exists(savepathR):
            os.mkdir(savepathR)
        for savename in allCTlist_dict.keys():
            plotNeighborComposition(labels,allCTlist_dict[savename],adjCluster,allDist,distThresh,savepathR,savename)

def plotNDBscan(allCTlist_dict,adjCluster,allDist,distThresh,savepath,epsL,min_samplesL,n_pcs):
    for eps in epsL:
        for min_samples in min_samplesL:
            savenamecluster='dbscan_eps'+str(eps)+'msamples'+str(min_samples)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            savepathR=os.path.join(savepath,savenamecluster)
            if not os.path.exists(savepathR):
                os.mkdir(savepathR)
            for savename in allCTlist_dict.keys():
                plotNeighborComposition(labels,allCTlist_dict[savename],adjCluster,allDist,distThresh,savepathR,savename)
                
def plotNAgg(allCTlist_dict,adjCluster,allDist,distThresh,savepath,nclusterL,aggmetricL,n_pcs):
    for ncluster in nclusterL:
        for aggmetric in aggmetricL:
            savenamecluster='agg_ncluster'+str(ncluster)+aggmetric+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            savepathR=os.path.join(savepath,savenamecluster)
            if not os.path.exists(savepathR):
                os.mkdir(savepathR)
            for savename in allCTlist_dict.keys():
                plotNeighborComposition(labels,allCTlist_dict[savename],adjCluster,allDist,distThresh,savepathR,savename)
            
def plotNMinibatchKmean(allCTlist_dict,adjCluster,allDist,distThresh,savepath,nclusterL,n_pcs):
    for ncluster in nclusterL:
        savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        savepathR=os.path.join(savepath,savenamecluster)
        if not os.path.exists(savepathR):
            os.mkdir(savepathR)
        for savename in allCTlist_dict.keys():
            plotNeighborComposition(labels,allCTlist_dict[savename],adjCluster,allDist,distThresh,savepathR,savename)

In [ ]:
# separate plots by region and cell types
for s in plot_samples.keys():
# for s in ['control8']:
#     if s in ['disease13']:
#         continue
    print(s)
    sampleidx=plot_samples[s]
    adj_s=adj_list[s].todense()
    celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
    celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
    region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
    allDist=euclidean_distances(sobj_coord_np,sobj_coord_np)
    minDist=np.min(allDist[allDist != 0])
    maxDist=np.max(allDist[allDist != 0])
    distThresh=np.concatenate((np.copy(distThreshAsMin)*minDist,[maxDist]))
    
    allCTlist_dict={'celltype_broad':celltype_broad,'celltype_sub':celltype_sub}
    origCT=np.unique(celltype_broad)
    celltypeplot=np.concatenate((origCT,list(combineCelltype.keys())),axis=None)
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr

        if inverseAct:
            samplename+='_beforeAct'
        sampledir=os.path.join(plotsavepath,samplename)
        if not os.path.exists(sampledir):
            os.mkdir(sampledir)
            
        for r in np.unique(region):
#             if r!='Hippocampus':
#                 continue
            print(r)
            ridx=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']==r
            for reg in celltypeplot:
                if not ((r=='Cortex' and reg in ['Ex']) or (r=='Hippocampus' and reg in ['CA1','DG','Micro','CA'])):
#                 if reg!='CA1':
                    continue
#                 if s=='control8' and r =='Hippocampus' and reg in ['Astro','CA1','CA2','CA3','DG','Endo','Ex','Inhi','LHb','Micro','OPC']:
#                     continue
                print(reg)
                clustersavedir=os.path.join(plotsavepath,samplename,'cluster'+'_'+reg,r)
                if not os.path.exists(clustersavedir):
                    print('DNE: '+clustersavedir)
                    continue
                plotNsavedir=os.path.join(plotsavepath,samplename,'neighborhood'+'_'+reg+r)
                if not os.path.exists(plotNsavedir):
                    os.mkdir(plotNsavedir)

                if reg in origCT:
                    ct_idx=celltype_broad==reg
                else:
                    ct_idx=False
                    for i in combineCelltype[reg]:
                        ct_idx=np.logical_or(ct_idx,celltype_broad==i)
                
                reg_idx=np.logical_and(ridx,ct_idx)
                
                        
                if np.sum(reg_idx)<minCells:
                    continue
                if 'leiden' in clustermethod:
                    plotNLeiden(allCTlist_dict,adj_s[reg_idx],allDist[reg_idx],distThresh,plotNsavedir,n_neighbors,n_pcs,min_dist,resolution)
                if 'dbscan' in clustermethod:
                    plotNDBscan(allCTlist_dict,adj_s[reg_idx],allDist[reg_idx],distThresh,plotNsavedir,epslist,min_sampleslist,n_pcs)
                if 'agglomerative' in clustermethod:
                    plotNAgg(allCTlist_dict,adj_s[reg_idx],allDist[reg_idx],distThresh,plotNsavedir,nclusterlist,aggMetric,n_pcs)
                if 'kmeanbatch' in clustermethod:
                    plotNMinibatchKmean(allCTlist_dict,adj_s[reg_idx],allDist[reg_idx],distThresh,plotNsavedir,nclusterlist,n_pcs)

disease13
Cortex
Ex
DG
Micro
CA


In [63]:
np.where(np.unique(celltype_broad)=='Astro')[0][0]

0